## 8. flask 와 데이터베이스
- 실제 웹서비스에서는 다양한 데이터베이스를 사용함
- 변경 여지가 적은 경우, mysql 과 같은 관계형 데이터베이스 사용
  - 가장 정보가 많고, 수십년간 사용되었기 때문에 보안, 안정적임
- 변경 여지가 큰 웹서비스 로깅(logging)을 위해서는 NoSQL 을 사용
- 이외에 웹서비스 성능 향상을 위해 redis 와 같은 in-memory 방식의 데이터베이스도 많이 사용됨

> 기존 강좌에서 다뤘던 mysql(SQL) 과 mongodb(NoSQL) 에 대해 간단한 웹서비스 구현을 위한 사전 작업과 함께, 기존 강좌의 핵심 내용 정리

### 블로그 기능과 데이터베이스 활용
- mysql: 구독으로 얻어진 이메일 주소 정보를 저장
- mongodb: 블로그 접근 사용자의 IP 와 접근 시간을 로그 형태로 저장 
  - logging 라이브러리로 파일로 저장하지 않고, 몽고db를 활용
  - 몽고db가 익숙치 않을 경우, logging 라이브러리로도 활용 가능
  - 로깅은 통상적으로 수시로 포맷이 바뀌는 경우가 많으므로, 스키마 설정이 필요없는 NoSQL 이 적합

## 8.1. flask 와 MySQL 사용
- mongodb 는 크롤링 데이터등과 같이 방대하지만, 일부 유실되도 큰 문제없는 데이터 처리시 더 적합
- 규격화되고 각 데이터의 극도의 안정성이 필요한 데이터는 MySQL 이 더 적합

### SQLAlchemy VS Pymysql
- Django 등에서 다양한 데이터베이스를 객체로 만들어서, 사용법을 통일하는 기법으로 SQLAlchemy 라는 기능을 사용함
- SQL 이외에 추가적인 SQLAlchemy 기반 CRUD 기법을 익혀야 함
- SQL, mongodb CRUD 에 익숙해지는 것이 보다 중요함


> 우리는 pymysql 을 쓰기로 함

- 굳이 객체로 만들어서, 복잡하게 다룰 필요 없음
- 객체 생성이 느리므로, 성능도 직접 데이터베이스를 엑세스하는 것에 비해 나을 것이 없음

```
At the ORM level, the speed issues are because creating objects in Python is slow, and the SQLAlchemy ORM applies a large amount of bookkeeping to these objects as it fetches them, which is necessary in order for it to fulfill its usage contract, including unit of work, identity map, eager loading, collections, etc.
```

### Mysql 로컬 설치
- 윈도우
  - MySQL Community Server Download 로 검색 후, 다운로드 및 설치
- 맥
  - brew 설치 후,
  
```
brew list
brew install mysql
mysql.server start
mysql -u root
```

### Mysql 접속 및 설정
- 터미널 모드에서 다음과 같이 명령
```
mysql -u root
GRANT ALL PRIVILEGES ON *.* TO 'root'@'localhost';
CREATE USER 'dave'@'localhost' IDENTIFIED BY 'funcoding';
GRANT ALL PRIVILEGES ON *.* TO 'dave'@'localhost';
flush privileges;
```

### 참고: Ubuntu 에서 MySQL 설치

1. <b>sudo apt-get update</b>
2. <b>sudo apt-get install mysql-server</b>
3. <b>sudo mysql_secure_installation</b>
   - 비밀번호 설정 필요
4. <b>sudo vim /etc/mysql/mysql.conf.d/mysqld.cnf</b>
   > 참고: 만약 vi 명령이 없다면, 다음과 같이 vi 설치 및 사용법 숙지
   > sudo apt-get install vim

   - 다음 설정을 [mysqld] 에 추가 ([mysqld] 설정 예 참고)
   1. mysql 한글 설정 추가
      ```bash
      collation-server = utf8_unicode_ci
      character-set-server = utf8
      skip-character-set-client-handshake
      ```
   2. mysql 원격 접속 허용
      - bind-address 0.0.0.0 로 변경
      
   **/etc/mysql/mysql.conf.d/mysqld.cnf 파일의 [mysqld] 설정 예**
   
```bash
[mysqld]
#
# * Basic Settings
#
user            = mysql
pid-file        = /var/run/mysqld/mysqld.pid
socket          = /var/run/mysqld/mysqld.sock
port            = 3306
basedir         = /usr
datadir         = /var/lib/mysql
tmpdir          = /tmp
lc-messages-dir = /usr/share/mysql
skip-external-locking
collation-server = utf8_unicode_ci
character-set-server = utf8
skip-character-set-client-handshake
#
# Instead of skip-networking the default is now to listen only on
# localhost which is more compatible and is not less secure.
bind-address            = 0.0.0.0
```

5. <b>sudo systemctl start mysql.service</b>
 - mysql 서비스 시작
참고:
    
```bash
Usage: sudo systemctl start|stop|restart|status mysql.service 
```

6. <b>모든 호스트에서 mysql 접속 허용하기</b>
 
```bash
 # sudo mysql 로 mysql 터미널로 들어감
GRANT ALL PRIVILEGES ON *.* TO 'root'@'%';
CREATE USER 'dave'@'%' IDENTIFIED BY 'funcoding';
CREATE DATABASE dave_db;
GRANT ALL PRIVILEGES ON dave_db.* TO 'dave'@'%' IDENTIFIED BY '암호';
flush privileges;
exit
 # sudo systemctl restart mysql.service
```


### pymysql 라이브러리 소개 및 설치
* mysql을 python에서 사용할 수 있는 라이브러리 (pymysql 라이브러리 이외에도 MySQLdb(Mysql-pytion), MySQL connector 등 다양한 라이브러리 존재)
* 이 중에서 설치가 가장 쉬운 라이브러리
* 설치
  - pip install PyMySQL
  
* 일반적인 mysql 핸들링 코드 작성 순서
  1. PyMySql 모듈 import
  2. pymysql.connect() 메소드를 사용하여 MySQL에 연결
     - 호스트명, 포트, 로그인, 암호, 접속할 DB 등을 파라미터로 지정
  3. MySQL 접속이 성공하면, Connection 객체로부터 cursor() 메서드를 호출하여 Cursor 객체를 가져옴
  4. Cursor 객체의 execute() 메서드를 사용하여 SQL 문장을 DB 서버에 전송
  5. SQL 쿼리의 경우 Cursor 객체의 fetchall(), fetchone(), fetchmany() 등의 메서드를 사용하여 서버로부터 가져온 데이타를 코드에서 활용
  6. 삽입, 갱신, 삭제 등의 DML(Data Manipulation Language) 문장을 실행하는 경우, INSERT/UPDATE/DELETE 후 Connection 객체의 commit() 메서드를 사용하여 데이타를 확정
  7. Connection 객체의 close() 메서드를 사용하여 DB 연결을 닫음

### Pymysql 로 mysql 접속
* pymysql.connect() 메소드를 사용하여 MySQL에 연결
     - 호스트명, 포트, 로그인, 암호, 접속할 DB 등을 파라미터로 지정
     - 주요 파라미터
       - host : 접속할 mysql server 주소
       - port : 접속할 mysql server 의 포트 번호
       - user : mysql ID
       - passwd : mysql ID의 암호
       - db : 접속할 데이터베이스
       - charset='utf8' : mysql에서 select하여 데이타를 가져올 때 한글이 깨질 수 있으므로 연결 설정에 넣어줌

### 사전 작업
```
mysql -u root
CREATE DATABASE blog_db;
```

In [4]:
import pymysql

db_conn = pymysql.connect(
        host='localhost', 
        port=3306, 
        user='dave', 
        passwd='funcoding', 
        db='blog_db', 
        charset='utf8')

### 사용자 테이블
- USER_ID 로 사용자 정보를 접근
- USER_EMAIL 도 unique 해야 하지만 동일한 USER_EMAIL 은 입력 코드에서 체크하기로 함

```sql
CREATE TABLE user_info (
    USER_ID INT UNSIGNED NOT NULL AUTO_INCREMENT,
    USER_EMAIL VARCHAR(100) NOT NULL,
    BLOG_ID CHAR(4),
    PRIMARY KEY(USER_ID)
);
```

### mysql 테스트

> 실제 flask 코드에 넣어서 테스트할 경우, 테스트에 굉장한 시간 소요
> 필요한 테스트는 주피터 노트북상에서 가능한 많이 한 후에, 최종 결과가 될 수 있는 코드만 flask 코드로 변환

In [10]:
import pymysql

db_conn = pymysql.connect(
        host='localhost', 
        port=3306, 
        user='dave', 
        passwd='funcoding', 
        db='blog_db', 
        charset='utf8')

dave_db = db_conn.cursor()
dave_db

### 기존 테이블 수 확인하기

In [6]:
sql = 'SHOW TABLES;'
dave_db.execute(sql)

1

In [ ]:
sql = """
CREATE TABLE user_info (
    USER_ID INT UNSIGNED NOT NULL AUTO_INCREMENT,
    USER_EMAIL VARCHAR(100) NOT NULL,
    BLOG_ID CHAR(4),
    PRIMARY KEY(USER_ID)
);
"""
dave_db.execute(sql)
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

### 테이블 수 확인하기
- 테이블 생성되었으므로 테이블 수가 1이어야 맞음

In [7]:
sql = 'SHOW TABLES;'
dave_db.execute(sql)

1

### 테이블 삭제하기

In [8]:
sql = 'DROP TABLE user_info;'
dave_db.execute(sql)
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

### 테이블 수 확인하기
- 테이블 생성되었으므로 테이블 수가 1이어야 맞음

In [9]:
sql = 'SHOW TABLES;'
dave_db.execute(sql)

0

### 테이블 재생성하기

In [10]:
sql = """
CREATE TABLE user_info (
    USER_ID INT UNSIGNED NOT NULL AUTO_INCREMENT,
    USER_EMAIL VARCHAR(100) NOT NULL,
    BLOG_ID CHAR(4),
    PRIMARY KEY(USER_ID)
);
"""
dave_db.execute(sql)
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

### 테이블에 데이터 넣기

In [11]:
user_email = 'test@test.com'
blog_id = 'A'

In [12]:
sql = "INSERT INTO user_info (USER_EMAIL, BLOG_ID) VALUES ('%s', '%s')" % (str(user_email), str(blog_id))
dave_db.execute(sql)
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

### 테이블 데이터 조회(SELECT)
  - Cursor Object 가져오기: cursor = db.cursor()  
  - SQL 실행하기: cursor.execute(SQL)
  - mysql 서버로부터 데이터 가져오기: fetch 메서드 사용
    - fetchall(): Fetch all the rows
    - fetchmany(size=None): Fetch several rows
    - fetchone(): Fetch the next row

In [11]:
sql = "SELECT * FROM user_info"
dave_db.execute(sql)
results = dave_db.fetchall()
for result in results:
    print (result, type(result))

(2, 'jhleeroot@gmail.com', 'A') <class 'tuple'>
(3, 'dave@gmail.com', 'A') <class 'tuple'>
(4, 'dream@fun-coding.org', 'A') <class 'tuple'>


### USER_EMAIL 로 테이블 데이터 조회

In [2]:
user_id = 1

In [5]:
sql = "DELETE FROM user_info WHERE USER_ID = %d" % (user_id)
print(dave_db.execute(sql))
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

0


### 데이터베이스 연결 해제

In [15]:
db_conn.close()

### 참고: 데이터베이스 데이터 삭제하기
- 강좌 후반부 실제 구현시 관련 내용을 영상으로 보여드림

In [13]:
import pymysql

db_conn = pymysql.connect(
        host='localhost', 
        port=3306, 
        user='dave', 
        passwd='funcoding', 
        db='blog_db', 
        charset='utf8')

dave_db = db_conn.cursor()
dave_db

In [17]:
sql = "SELECT * FROM user_info"
dave_db.execute(sql)
results = dave_db.fetchall()
for result in results:
    print (result, type(result))

(2, 'jhleeroot@gmail.com', 'A') <class 'tuple'>
(3, 'dave@gmail.com', 'A') <class 'tuple'>


In [18]:
user_id = 4

In [19]:
sql = "DELETE FROM user_info WHERE USER_ID = %d" % (user_id)
print(dave_db.execute(sql))
db_conn.commit() # 데이터베이스를 변경하는 명령은 commit() 해주기

0


## 8.2. Mongodb 설치 
- MAC 환경
  - https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/
  
  ```
  brew install mongodb-community
  brew services start mongodb-community
  ```
  또는
  ```
  brew services restart mongodb-community
  ```
  
- 윈도우 환경
  - https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/
  
> 윈도우 환경에서 설치 및 실행 방법은 mongodb 강의에서 해당 강좌는 무료로 오픈해놓았으니 수강하셔서 확인 가능

### Mongodb 연결

In [1]:
import pymongo

username = ''
password = ''
ip_address = 'localhost'
connection = pymongo.MongoClient()
connection = pymongo.MongoClient('mongodb://%s' % (ip_address))
# connection = pymongo.MongoClient('mongodb://%s:%s@%s' % (username, password, ip_address))
blog_session_db = connection.blog_session_db
blog_ab = blog_session_db.blog_ab

### Mongodb 연결 확인
- mongodb 연결을 한번 해놓으면, 이론적으로는 해당 객체를 사용, 몽고db CRUD 를 실행하면 됨
- 하지만, 실제로는 mongodb 가 다양한 원인으로 다운되거나, 연결이 해제되는 경우가 있음
  - 이 경우, 연결이 되어있음을 가정하고, CRUD를 실행할 경우 에러가 남
- 따라서 연결된 객체가 아직 mongodb 에 연결이 되어있는지 체크하는 방법이 필요함
  - 물론, 이 방법도 mongodb 가 아예 다운된 경우등을 체크하도록 코드를 더 정교하게 작성할 수도 있지만,
  - 문제를 mongodb 는 다운되더라도 재기동된다고 가정하고, (docker 등 다른 시스템을 통해 해당 기능 활성화)
  - 연결이 해제된 경우만 체크하는 기법을 사용 
    - 특별한 방법이 없으므로, 가장 간단한 명령을 내려보는 방법으로 진행

In [20]:
connection.admin.command('ismaster')

{'ismaster': True,
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2020, 8, 15, 7, 25, 55, 594000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 61,
 'minWireVersion': 0,
 'maxWireVersion': 8,
 'readOnly': False,
 'ok': 1.0}

In [21]:
connection.server_info()

{'version': '4.2.8',
 'gitVersion': '43d25964249164d76d5e04dd6cf38f6111e21f5f',
 'modules': [],
 'allocator': 'system',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 2, 8, 0],
 'openssl': {'running': 'Apple Secure Transport'},
 'buildEnvironment': {'distmod': '',
  'distarch': 'x86_64',
  'cc': '/Applications/Xcode10.2.0.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/clang: Apple LLVM version 10.0.1 (clang-1001.0.46.3)',
  'ccflags': '-isysroot /Applications/Xcode10.2.0.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX10.14.sdk -mmacosx-version-min=10.12 -target darwin16.0.0 -arch x86_64 -fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-unused-private-field -Wno-deprecated-declarations -Wno-tautological-constant-out-of-range-compare -Wno-tautological-constant-compare -Wno-tautological-u

### INSERT

In [22]:
blog_ab.insert_one({'emailid':'jhleeroot@gmail.com'})

### SELECT

In [23]:
blog_ab.find_one( {'emailid':'jhleeroot@gmail.com'} )

{'_id': ObjectId('5f34e53a2171fe53b403d027'), 'emailid': 'jhleeroot@gmail.com'}

### DELETE

In [24]:
blog_ab.delete_one( {'emailid':'jhleeroot@gmail.com'} )

### SELECT *

In [3]:
blog_logs = blog_ab.find()
for log in blog_logs:
    print(log)

{'_id': ObjectId('5f36362bd30e2ec7d748d6e9'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'access_time': '14/08/2020 15:58:51'}
{'_id': ObjectId('5f36362bd30e2ec7d748d6eb'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'access_time': '14/08/2020 15:58:51'}
{'_id': ObjectId('5f363660d30e2ec7d748d6ed'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'access_time': '14/08/2020 15:59:44'}
{'_id': ObjectId('5f36366dd30e2ec7d748d6ef'), 'session_ip': '127.0.0.1', 'user_email': 'jhleeroot@gmail.com', 'access_time': '14/08/2020 15:59:57'}
{'_id': ObjectId('5f36372b043800e068a3edb9'), 'session_ip': '127.0.0.1', 'user_email': 'anonymous', 'page': 'blog_A.html', 'access_time': '14/08/2020 16:03:07'}
{'_id': ObjectId('5f36372f043800e068a3edbb'), 'session_ip': '127.0.0.1', 'user_email': 'jhleeroot@gmail.com', 'page': 'blog_A.html', 'access_time': '14/08/2020 16:03:11'}
{'_id': ObjectId('5f363734043800e068a3edbd'), 'session_ip': '127.0.0.1', 'user_email': 'jhleeroot@gmail.co